In [82]:
import cv2
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
%matplotlib inline

In [83]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.utils.np_utils import to_categorical

In [84]:
dataset = 'mag-lev-images'
size = 64

In [85]:
def class_label(img):
    if '100-lido' in img:
        lbl = 0
    elif '100-caff' in img:
        lbl = 1
    elif '50-50-lido-caff' in img:
        lbl = 2
    return lbl

In [86]:
def images_with_label():
    images = []
    for i in tqdm(os.listdir(dataset)):
        path = os.path.join(dataset, i)
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, (size, size))
            images.append([np.array(img), class_label(i)])
    return images

In [87]:
images = images_with_label()

X = np.array([i[0] for i in images]).reshape(-1, size, size, 1)
y = np.array([i[1] for i in images])

100%|██████████| 29/29 [00:00<00:00, 80.23it/s]


In [97]:
kfold = StratifiedKFold(n_splits=7, shuffle=True)
cvscores = []

In [98]:
for train, test in kfold.split(X, y):    
    model = Sequential()

    model.add(InputLayer(input_shape=[size,size,1]))
    model.add(Conv2D(filters=32,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Conv2D(filters=50,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Conv2D(filters=80,kernel_size=5,strides=1,padding='same',activation='relu'))
    model.add(MaxPool2D(pool_size=5,padding='same'))

    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(512,activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(3,activation='softmax'))
    optimizer = Adam(lr=1e-2)

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',metrics=['accuracy'])
    model.fit(x=X[train],y=to_categorical(y[train]),epochs=50,batch_size=10,verbose=0)
    model.summary()
    
    # evaluate the model
    scores = model.evaluate(X[test], to_categorical(y[test]), verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    cvscores.append(scores[1] * 100)
    print("\n\n")

Model: "sequential_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_118 (Conv2D)          (None, 64, 64, 32)        832       
_________________________________________________________________
max_pooling2d_118 (MaxPoolin (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 13, 13, 50)        40050     
_________________________________________________________________
max_pooling2d_119 (MaxPoolin (None, 3, 3, 50)          0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 3, 3, 80)          100080    
_________________________________________________________________
max_pooling2d_120 (MaxPoolin (None, 1, 1, 80)          0         
_________________________________________________________________
dropout_79 (Dropout)         (None, 1, 1, 80)        

accuracy: 25.00%



Model: "sequential_45"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_133 (Conv2D)          (None, 64, 64, 32)        832       
_________________________________________________________________
max_pooling2d_133 (MaxPoolin (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_134 (Conv2D)          (None, 13, 13, 50)        40050     
_________________________________________________________________
max_pooling2d_134 (MaxPoolin (None, 3, 3, 50)          0         
_________________________________________________________________
conv2d_135 (Conv2D)          (None, 3, 3, 80)          100080    
_________________________________________________________________
max_pooling2d_135 (MaxPoolin (None, 1, 1, 80)          0         
_________________________________________________________________
dropout_89 (Dropout)         (Non

In [99]:
print("Average Accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

Average Accuracy: 32.14% (+/- 11.29%)
